# Retrieving, Processing, and Visualizing Deterministic Precipitation ICON Forecast Data

This notebook demonstrates the full workflow for accessing, processing, and visualizing determinstic precipitation forecast data from the ICON numerical weather model. The data is provided by MeteoSwiss as part of Switzerland’s Open Government Data (OGD) initiative.

The core functionality is powered by the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — developed in-house to simplify working with numerical weather model data. It includes the `ogd_api` module for data access, along with utilities for applying temporal and spatial operators to model output, such as deaccumulation and regridding.

For visualization, this notebook uses the [earthkit-plots](https://earthkit-plots.readthedocs.io/en/latest/examples/guide/01-introduction.html) library developed by ECMWF, which offers intuitive plotting tools for meteorological data.

---

## 🔍 **What You’ll Do in This Notebook**

 🛰️  **Retrieve**  
    Fetch deterministic ICON-CH2-EPS forecast data (e.g., total precipitation) via the `ogd_api` module.

 🧪  **Deaggregate**  
    Convert precipitation accumulated over the total forecast duration into hourly values.

 🧭  **Regrid**  
    Interpolate ICON data from its native, icosahedral grid to a regular latitude/longitude grid.

 🌍  **Visualize**  
    Plot the processed data on a map with meteorological styling using [earthkit-plots](https://earthkit-plots.readthedocs.io/en/latest/examples/guide/01-introduction.html).

---

## Retrieving Precipitation Forecast
In this first part, we retrieve precipitation forecast data from the ICON numerical weather model. To access this data, we use the `ogd_api` module from the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — a convenient interface for accessing numerical weather forecasts via the [STAC (SpatioTemporal Asset Catalog) API](https://data.geo.admin.ch/api/stac/static/spec/v1/apitransactional.html#tag/Data/operation/getAsset), which provides structured access to Switzerland’s open geospatial data.

#### 📁  Browsing the STAC Catalog (Optional)

If you'd like to explore the ICON forecast datasets interactively before writing code, you can browse them directly in the STAC catalog:

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH1-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch1?.language=en)

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH2-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch2?.language=en)


Below is a screenshot of the ICON-CH2-EPS collection as seen in the STAC browser interface.


![browser-ch2-resized.PNG](./images/browser-ch2-resized.PNG)

### Creating Requests
To retrieve the forecast data, we first define a set of API requests using the `ogd_api.Request` class.
Each request targets a different lead time, which is the elapsed time since the forecast's start time (reference or initialization datetime).
In this example, we retrieve hourly precipitation accumulations (`TOT_PREC`) for three consecutive lead times: +0h, +1h and +2h. Each hourly forecast value includes all precipitation from the reference time up to the corresponding lead time.

>⏰ **Forecast Availability**: Forecast data will typically be available a couple of hours after the reference time — due to the model runtime and subsequent upload time. The data remains accessible for 24 hours after upload.

In [ ]:
from datetime import datetime, timezone
from meteodatalab import ogd_api

amount_of_lead_times = 3
reqlist = []

# Set default date of today midnight in UTC
today_midnight_utc = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)

for i in range(amount_of_lead_times):
    req = ogd_api.Request(
        collection="ogd-forecasting-icon-ch2",
        variable="TOT_PREC",
        reference_datetime=today_midnight_utc,
        perturbed=False,
        horizon=f"P0DT{i}H",
    )
    reqlist.append(req)

Each argument in the request serves the following purpose:

| Argument             | Description |
|----------------------|-------------|
| `collection`         | Forecast collection to use (e.g., `ogd-forecasting-icon-ch2`). |
| `variable`           | Meteorological variable of interest (`TOT_PREC` = total precipitation). |
| `reference_datetime` | Initialization time of the forecast in **UTC**, provided as either:<br>- [datetime.datetime](https://docs.python.org/3/library/datetime.html#datetime-objects) object (e.g.,<br> &nbsp; `datetime.datetime(2025, 5, 22, 9, 0, 0, tzinfo=datetime.timezone.utc)`) <br>- [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Combined_date_and_time_representations) date string (e.g., `"2025-05-22T09:00:00Z"`)|
| `perturbed`          | If `True`, retrieves ensemble forecast members; if `False`, returns the deterministic forecast. |
| `horizon`            | Forecast lead time, provided as either:<br>– [datetime.timedelta](https://docs.python.org/3/library/datetime.html#timedelta-objects) object (e.g., `datetime.timedelta(hours=2)`) <br>– [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601#Durations) duration string (e.g., `"P0DT2H"`)|

### Retrieving Data
We now send our list of requests to the API and retrieve the resulting datasets using the `get_from_ogd()` function.
Each response is returned as an **[xarray.DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html)**, which is efficient for handling multi-dimensional data.

> 💡 **Tip**: Use temporary caching with earthkit-data to skip repeated downloads — it's auto-cleaned after the session.
> *For more details, see the [earthkit-data caching docs](https://earthkit-data.readthedocs.io/en/latest/examples/cache.html)*.

> 💡 **Hint**: If you get an error message containing "HTTPError: 403 Client Error: Forbidden for url", you may be trying to retrieve data older than 24h hours! Please adjust your requests.

In [ ]:
from earthkit.data import config
config.set("cache-policy", "temporary")

da_list = []
for req in reqlist:
    da = ogd_api.get_from_ogd(req)
    da_list.append(da)

## Data Preprocessing

Once the data has been loaded, continue with the preprocessing of the data if needed. The following preprocessing steps
are described here:
- Merging the data array containing the different lead times into one array
- Disaggregating the accumulated precipitation data
- Regridding the data to a regular latitude/longitude grid

### Merging Lead Times
To work with a continuous time series, we concatenate forecast data for different lead times into a single `xarray.DataArray`.

In [ ]:
import xarray as xr
tot_prec = xr.concat(da_list, dim="lead_time")
tot_prec

The resulting `xarray.DataArray` has the following dimensions:

- `eps` (ensemble members): 1 member (for deterministic data)
- `ref_time`: Single reference time (e.g. default date of today midnight)
- `lead_time`: 3 hourly intervals
- `cell`: 283,876 spatial grid points

It includes attributes specifying some meta data such as:

- `parameter`: information on the meteorogical parameter, here total precipitation
- `vcoord_type`: information of the vertical coordinate type, here surface field

> &#x1F525; **Expert users:** If you are interested in detailed meta data information, i.e. the GRIB encodings, you can retrieve it from the `metadata` attribute.
Please refer to the [earthkit-data metadata documentation](https://earthkit-data.readthedocs.io/en/latest/examples/metadata.html).

### Disaggregating Precipitation
Convert precipitation accumulated over the whole forecast time into one-hour precipitation by differencing consecutive lead times. This is done using the `delta()` function from [meteodatalab](https://meteoswiss.github.io/meteodata-lab/), which simplifies the operation.

In [ ]:
from meteodatalab.operators import time_operators as time_ops
import numpy as np

prec_1h = time_ops.delta(tot_prec, np.timedelta64(1,"h"))

For the next steps, we'll work with a **single lead time** from the forecast.
Xarray's `.isel()` method lets us easily select along a specific dimension — here, we use it to pick one lead time.


In [ ]:
prec = prec_1h.isel(lead_time=2)

### Regridding to a Regular Grid

To visualize or compare ICON model outputs with other datasets, it is often necessary to regrid the data from its native unstructured grid to a regular latitude/longitude grid.

This can be done using the `regrid` utilities provided by [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/).

We begin by defining a **target grid**, starting with its **spatial extent**, also known as the bounding box. This is specified by the minimum and maximum coordinate values along the x and y axes (commonly referred to as longitude and latitude when using a geographic CRS such as [WGS84 EPSG:4326](https://epsg.io/4326)). Note that in projected coordinate systems, these values are not technically “longitude” and “latitude,” but rather x and y coordinates in the chosen CRS.

In this example, we use the `regrid.iconremap` function to perform the regridding. Depending on how the target grid's spatial extent is defined, it can fall into one of the following cases:
  - ✅ **Fully overlap** the ICON-CH1/CH2-EPS domain: all target grid points are interpolated.
  - ⚠️ **Partially overlap**: only the overlapping area will contain valid values; other points are filled with `NaN`.
  - ❌ **Lie completely outside** the ICON-CH1/CH2-EPS domain: all target grid values will be `NaN`.

📌 In our example, we define the spatial extent to fully match the **ICON-CH1/CH2-EPS** domain.

<p align="center">
  <img src="images/icon-ch1-ch2-eps-domain.png" alt="ICON domain illustration" width="450"/>
</p>

In [ ]:
# Define the spatial extent of the target grid
# (we retain the original ICON-CH1/CH2-EPS domain)
xmin, xmax = -0.817, 18.183   # Longitude bounds
ymin, ymax = 41.183, 51.183   # Latitude bounds


Next, we define the **number of grid points** (`nx`, `ny`) to control the resolution of the regular grid.
    
To match the resolution of the ICON-CH2-EPS domain (≈ **2 km**), we define the number of points directly:


In [ ]:
# Compute the number of grid points to match ~2 km resolution
nx, ny = 732, 557  # Number of grid points in x and y

> 🔢 [How to compute nx, ny for a custom resolution](./how_to_compute_number_of_grid_points.md)

The parameters n<sub>x</sub> and n<sub>y</sub> are passed to `regrid.RegularGrid`, which defines the spatial geometry of the output grid.

In [ ]:
from rasterio.crs import CRS
from meteodatalab.operators import regrid

# Create a regular lat/lon grid using EPSG:4326
destination = regrid.RegularGrid(
    CRS.from_string("epsg:4326"), nx, ny, xmin, xmax, ymin, ymax
)

Finally, remap the data to the regular grid.


In [ ]:
prec_geo = regrid.iconremap(prec, destination)

## Visualizing Precipitation on a Map
We now visualize the 1-hour precipitation on a map. This uses [earthkit-plots](https://earthkit-plots.readthedocs.io/en/latest/examples/guide/01-introduction.html), the visualization component of [Earthkit](https://earthkit.readthedocs.io/en/latest/) — a suite of tools designed to simplify access, processing, and visualization of weather and climate data in Python.

In [ ]:
from earthkit.plots.geo import domains
import earthkit
import pandas as pd

# Define the map domain using the regridded data's bounding box
domain = domains.Domain.from_bbox(
    bbox=[xmin, xmax, ymin, ymax],
    name="CH2"
)

# Create the map instance
chart = earthkit.plots.Map(domain=domain)

# Plot the regridded precipitation field
chart.grid_cells(prec_geo, x="lon", y="lat")

# Add standard map features
chart.land()
chart.coastlines()
chart.borders()
chart.gridlines()

# Extract forecast metadata
ref_time = pd.to_datetime(prec_geo.coords["ref_time"].values[0]).strftime("%Y-%m-%d %H:%M UTC")
lead_ns = prec_geo.coords["lead_time"].values
lead_hours = pd.to_timedelta(lead_ns).total_seconds() / 3600
lead_time = f"{lead_hours:.0f}h"

# Prepare title and legend labels
parameter = prec_geo.attrs["parameter"]
title = f"Hourly Precipitation | Ref: {ref_time} | Lead Time: {lead_time}"
legend_label = f"{parameter['name']} ({parameter['units']})"

# Add title and legend to the map
chart.title(title)
chart.legend(label=legend_label)

# Display the map
chart.show()